# DSAP 108

In [1]:
import numpy as np
import pandas as pd 
from datetime import datetime

### 讀取資料

In [2]:
grades = pd.read_csv('../../data/DSAP108 grades.csv')
v_info = pd.read_excel('../../data/DSAP108 videos_sep.xlsx', sheet_name = 'video info')
records = pd.read_excel('../../data/DSAP108 videos_sep.xlsx', sheet_name = 'watching records')
student_id = pd.read_excel('../../data/DSAP108 videos_sep.xlsx', sheet_name = 'student id')

### 觀看時間、觀看速度、平均觀看次數

In [3]:
### 觀看時間、觀看速度、平均觀看次數

# 開一個 table 存等一下計算的資料
student_info = pd.DataFrame(data = grades.drop(0)['student_id'])

# 計算總觀看時間
total_spent_time = [] # 總觀看時間
total_watch_video_time = [] # 觀看的總影片時間

for i in student_info['student_id']:
    watch_record_i = records[records['student_id'] == int(i)]
    # 抓出有在看的資料
    real_watch_i = watch_record_i[(watch_record_i['end'] > watch_record_i['start']) & (watch_record_i['playback_rate'] != 0)]
    video_time_i = real_watch_i['end'] - real_watch_i['start']# 觀看紀錄 的 影片時間(end - start)
    total_watch_video_time.append(sum(video_time_i))
    total_spent_time.append(sum(video_time_i/real_watch_i['playback_rate']))

student_info['total_watch_time'] = total_spent_time

# 計算撥放速度
total_spent_time = [x if x!= 0 else 1 for x in total_spent_time]
avg_playback_rate = [i/j for (i,j) in zip(total_watch_video_time, total_spent_time)]
student_info['avg_playback_rate'] = avg_playback_rate

# 計算影片平均觀看次數
length_l = list(v_info['meta'])# length of each lecture
'''
length_l = [( i.split('n=>')[1].split('}')[0]) for i in length_l]
length_l = [int(i) for i in length_l]
'''
total_length_l = sum(length_l)
student_info['avg_freq'] = student_info['total_watch_time']/(total_length_l/student_info['avg_playback_rate'])

### 拖曳次數、拖曳時間長度

In [4]:
forward_sec = []
backward_sec =[]
forward_times = []
backward_times =[]

# backwards
backward_records = records[(records['start'] > records['end']) & (records['playback_rate'] ==0)]
for i in student_info['student_id']:
    backward_records_i = backward_records[backward_records['student_id'] == int(i)]
    #display(backward_records_i)
    backward_times.append(len(backward_records_i))
    backward_sec.append(sum(backward_records_i['start'] - backward_records_i['end']))

#display(backward_records)

### Save data

In [5]:
student_info.to_excel('../../data/許哲/DSAP108.xlsx', index = False)